In [ ]:
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

In [ ]:
assert torch.cuda.is_available()

In [ ]:
!pip install nnunetv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 19.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

drive_dir = "/content/drive/MyDrive"
mount_dir = os.path.join(drive_dir, "Colab Notebooks")
base_dir = os.getcwd()

Mounted at /content/drive


In [ ]:
assert os.path.exists(drive_dir)
if os.path.exists(mount_dir) is False:
  os.makedirs(mount_dir)


In [ ]:
def make_if_dont_exist(data_path, overwrite = False):
  if os.path.exists(data_path):
    if not overwrite:
      print(f"{data_path} exists")
    else:
      print(f"{data_path} overwritten")
      shutil.rmtree(data_path)
      os.makedirs(data_path)
  else:
    os.makedirs(data_path)
    print(f"{data_path} created!")

In [ ]:
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw" : os.path.join(mount_dir, "nnUNet_raw"),
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"),
    "nnUNet_results" : os.path.join(mount_dir, "nnUNet_results"),
    "RAW_DATA_PATH" : os.path.join(mount_dir, "RawData")
}

for e_var, path in path_dict.items():
  os.environ[e_var] = path

for e_var, path in path_dict.items():
  if os.getenv(e_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(e_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(e_var)))
  make_if_dont_exist(path, overwrite = False)

  print("If no error occurred continue forward")

Current Working Directory /content
/content/drive/MyDrive/Colab Notebooks/nnUNet_raw exists
If no error occurred continue forward
/content/drive/MyDrive/Colab Notebooks/nnUNet_preprocessed exists
If no error occurred continue forward
/content/drive/MyDrive/Colab Notebooks/nnUNet_results exists
If no error occurred continue forward
/content/drive/MyDrive/Colab Notebooks/RawData exists
If no error occurred continue forward


In [ ]:
#Installation of base pretrained model framework based on which finetuned model weights are applied
os.chdir(path_dict["nnUNet_results"])
!nnUNetv2_install_pretrained_model_from_zip Dataset201_UKBB_GSTT_LAX_2Ch.zip
#!nnUNetv2_install_pretrained_model_from_zip Dataset202_UKBB_GSTT_LAX_4Ch.zip
#!rm "Dataset201_UKBB_GSTT_LAX_2Ch.zip"
os.chdir(base_dir)

In [ ]:
os.chdir(path_dict["nnUNet_results"])
!rm "Dataset201_UKBB_GSTT_LAX_2Ch.zip"
os.chdir(base_dir)

In [ ]:
#Custom trainer
trainer_code = """
from nnunetv2.training.nnUNetTrainer.nnUNetTrainer import nnUNetTrainer
import torch

class MyFineTuner(nnUNetTrainer):
  def __init__(self, plans: dict, configuration: str, fold: int, dataset_json:dict, device: torch.device = torch.device("cuda")):
    super().__init__(plans, configuration, fold, dataset_json, device)
    self.num_epochs = 100
"""

with open("/usr/local/lib/python3.11/dist-packages/nnunetv2/training/nnUNetTrainer/myfinetuner.py", "w") as f:
  f.write(trainer_code)

In [ ]:
#Model inference
!nnUNetv2_predict -i "${nnUNet_raw}/Dataset228_Heart4Ch/imagesTs" -o "${nnUNet_results}/Dataset228_Heart4Ch/predTs" -d 228 -c 2d -tr MyFineTuner


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

/usr/local/lib/python3.11/dist-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
There are 25 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we star